In [1]:
import numpy as np
import pandas as pd
import folium 

from folium.plugins import HeatMap
from gestion_donnees import data_pollution

In [2]:
QA=pd.read_csv("indices_QA_commune_IDF_04_2018.csv")


In [10]:
##GRAPHE DE TEST POUR LA HEATMAP
# Nombre d'entrées
n = 200

# Coordonnées centrées sur Paris
latitude_center = 48.8566
longitude_center = 2.3522

# Générer des coordonnées proches de Paris
latitudes = np.round(np.random.normal(loc=latitude_center, scale=0.01, size=n), 6)
longitudes = np.round(np.random.normal(loc=longitude_center, scale=0.01, size=n), 6)

# Générer des valeurs aléatoires de PM2.5 entre 1 et 100
pm25_values = np.random.randint(1, 101, size=n)

# Créer la structure attendue
data = [[lat, lon, pm] for lat, lon, pm in zip(latitudes, longitudes, pm25_values)]

# Créer le DataFrame sous la forme demandée
test = pd.DataFrame(data, columns=['Latitude', 'Longitude', 'PM2.5'])

#test=pd.DataFrame([[48.86,2.356,10],[48.90,2.35,40],[48.87,2.357,60],[48.88,2.359,90]],columns=['Latitude','Longitude','PM2.5'])
test.head()

,Latitude,Longitude,PM2.5
0,48.850147,2.359500,5
1,48.847631,2.349471,18
2,48.862059,2.361284,4
3,48.840896,2.346038,25
4,48.835355,2.350740,95


In [11]:
latitude = 48.8566
longitude = 2.3522
zoom=13
carte=folium.Map(location=[latitude,longitude],tiles="OpenStreetMap", zoom_start=zoom)

pollution = 42

#folium.Marker(location=[latitude,longitude], popup=f"Nox ={pollution} micro_g/m3", icon=folium.Icon(icon="cloud")).add_to(carte);

In [12]:
def niveau_couleur(value):
    if value <= 25 :
        return 'lightgreen'
    elif 25<value<=50 :
        return 'green'
    elif 50<value<=75 :
        return 'orange'
    else :
        return 'red'

In [6]:
#for index, row  in test.iterrows():
    #col=niveau_couleur(row['PM2.5'])
    #folium.Marker(location=[row['Latitude'],row['Longitude']], tooltip=f"Nox = {row['PM2.5']} micro_g/m3", icon=folium.Icon(icon="cloud",color=col)).add_to(carte);
    


In [13]:
HeatMap(test).add_to(carte)

In [8]:
carte.save("carte_pollution.html")

In [14]:
display(carte)